In [22]:
# import lib
import numpy as np
from numpy.random import rand
import matplotlib as plt
from sklearn.preprocessing import OneHotEncoder

In [23]:
# datasete
x=np.array(([0.9,0.8],[0.6,0.3],[0.9,0.1],[0.9,0.8],[0.2,0.35],[0.4,0.39]))
y=np.array((['Netural'],['Posative'],['Posative'],['Netural'],['Negative'],['Negative']))

In [24]:
# encode categoracal featuers as a one-hot numeric array
enc=OneHotEncoder(sparse=False,categories='auto')
y=enc.fit_transform(y.reshape(len(y),-1))

c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [25]:
enc.categories_

[array(['Negative', 'Netural', 'Posative'], dtype='<U8')]

In [26]:
y

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [33]:
# activation function
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

# softmax function
def softmax(x):
    expX=np.exp(x)
    return expX/expX.sum(axis=1,keepdims=True)

# Multi-layer feedforward neural network (2 hidden layers)
class NeuralNetwork:
    def __init__(self,x,y,nodes_in_layer1 = 4,nodes_in_layer2 = 3,nodes_in_layer3=3,l_rate=1):
        # defin x , y
        self.inputs_in_layer0 = x
        self.y = y
        self.l_rate = l_rate

        # definde and set the number of neurons in each layers
        self.nodes_in_layer1 = nodes_in_layer1
        self.nodes_in_layer2 = nodes_in_layer2
        self.nodes_in_layer3 = nodes_in_layer3
        
        # initialize weights and biases
        self.thetas_layer0 = np.random.rand(self.inputs_in_layer0.shape[1]+1,self.nodes_in_layer1)
        self.thetas_layer1 = np.random.rand(self.nodes_in_layer1+1,self.nodes_in_layer2)
        self.thetas_layer2 = np.random.rand(self.nodes_in_layer2+1,nodes_in_layer3)

    def feedforward(self):
        n = self.inputs_in_layer0.shape[0]
        self.Z1 = self.thetas_layer0[0] + np.dot(self.inputs_in_layer0,self.thetas_layer0[1:])
        self.layer1 = sigmoid(self.Z1)

        self.Z2 = self.thetas_layer1[0] + np.dot(self.layer1 , self.thetas_layer1[1:])
        self.layer2 = sigmoid(self.Z2)

        self.Z3 = self.thetas_layer2[0] + np.dot(self.layer2 , self.thetas_layer2[1:])
        self.layer3 = softmax(self.Z3)

        return self.layer3
    
    def cost_func(self):
        self.n = self.inputs_in_layer0.shape[0]
        self.cost = (1/self.n) * np.sum(-self.y * np.log(self.layer3))
        return self.cost
    
    def backprop(self):
        self.dE_dZ3 = (1/self.n)*(self.layer3 - self.y)
        self.dE_dtheta2 = np.dot(self.layer2.T,self.dE_dZ3)
        self.dE_dbias2 = np.dot(np.ones(self.n),self.dE_dZ3)

        # drivative of E with respect to theta and bias in layer1
        self.dE_dlayer2 = np.dot(self.dE_dZ3,self.thetas_layer2[1:].T)
        self.dE_dZ2 = np.multiply(self.dE_dlayer2,sigmoid(self.Z2)*(1-sigmoid(self.Z2)))
        self.dE_dtheta1 = np.dot(self.layer1.T,self.dE_dZ2)
        self.dE_dbias1 = np.dot(np.ones(self.n),self.dE_dZ2)

        # drivative of E with respect to theta and bias in layer0
        self.dE_dlayer1 = np.dot(self.dE_dZ2,self.thetas_layer1[1:].T)
        self.dE_dZ1 = np.multiply(self.dE_dlayer1,sigmoid(self.Z1)*(1-sigmoid(self.Z1)))
        self.dE_dtheta0 = np.dot(self.inputs_in_layer0.T,self.dE_dZ1)
        self.dE_dbias0 = np.dot(np.ones(self.n),self.dE_dZ1)

        # update theta using gradient descent in layer 2,1 and 0
        self.thetas_layer2[1:] = self.thetas_layer2[1:] - self.l_rate*self.dE_dtheta2
        self.thetas_layer1[1:] = self.thetas_layer1[1:] - self.l_rate*self.dE_dtheta1
        self.thetas_layer0[1:] = self.thetas_layer0[1:] - self.l_rate*self.dE_dtheta0

        # update bias using gradient descent in layer 2,1 and 0
        self.thetas_layer2[0] = self.thetas_layer2[0] - self.l_rate*self.dE_dbias2
        self.thetas_layer1[0] = self.thetas_layer1[0] - self.l_rate*self.dE_dbias1
        self.thetas_layer0[0] = self.thetas_layer0[0] - self.l_rate*self.dE_dbias0

        return self


In [34]:
NN = NeuralNetwork(x,y)
epochs = 500
losses = []
for i in range(epochs):
    predicted_output = NN.feedforward()
    error = NN.cost_func()
    losses.append(error)
    NN.backprop()
    print('itration #',i+1)
    print('Actual output: \n',y)
    print('Predicted output: \n',predicted_output)
    print('cost: \n',error,"\n")
    print('-------------------------------------------')

itration # 1
Actual output: 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Predicted output: 
 [[0.3127197  0.46977112 0.21750918]
 [0.31436937 0.46710456 0.21852607]
 [0.31422958 0.46740773 0.21836269]
 [0.3127197  0.46977112 0.21750918]
 [0.31516568 0.46573131 0.21910301]
 [0.31458826 0.46670295 0.21870878]]
cost: 
 1.1441024368845514 

-------------------------------------------
itration # 2
Actual output: 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Predicted output: 
 [[0.34841536 0.31827561 0.33330902]
 [0.34956562 0.3172067  0.33322768]
 [0.34947887 0.31744791 0.33307322]
 [0.34841536 0.31827561 0.33330902]
 [0.35010819 0.31652384 0.33336797]
 [0.34971122 0.31697352 0.33331526]]
cost: 
 1.0980263237107217 

-------------------------------------------
itration # 3
Actual output: 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Predicted output: 
 [[0.32905138 0.33816253 0.33278609]
 [0.33036984 0.